# Water Quality Analysis 

### Business Objective

This analysis aims to evaluate water quality across Maharashtra monitoring stations
and identify pollution hotspots, risk factors, and districts requiring intervention.

Key objectives:

• Assess overall water quality distribution

• Identify pollution indicators and violations

• Detect high-risk monitoring stations

• Analyze geographic pollution hotspots

• Validate relationships using statistical testing

• Provide actionable recommendations for MPCB

In [ ]:
#data importing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("NWMP_August2025_MPCB_0.csv",encoding="latin1")

# basic view
df.head()

: 

In [ ]:
#rows and columns
df.shape

In [ ]:
df.info()

In [ ]:
#statistical info about the data set
df.describe(include="all")

In [ ]:
df.columns

# About Data Features
## Feature Description

This dataset contains physical, chemical, biological, and environmental parameters used to assess the quality and pollution level of water bodies. These features help determine water usability for drinking, irrigation, and other purposes.

### Identification and Location Features
- **STN Code**: Unique identification code assigned to each monitoring station.
- **Sampling Date**: Date when the water sample was collected.
- **Sampling Time**: Time at which the sample was collected.
- **Month**: Month of sample collection.
- **Stn Name**: Name of the monitoring station.
- **Type Water Body**: Type of water body such as river, lake, or reservoir.
- **Name Of Water Body**: Name of the specific river, lake, or water source.
- **River Basin**: River basin to which the water body belongs.
- **District**: District where the monitoring station is located.
- **State Name**: State where the monitoring station is located.
- **latitude**: Geographic latitude coordinate of the station.
- **longitude**: Geographic longitude coordinate of the station.
- **Mon Agency**: Agency responsible for monitoring and collecting the data.
- **Frequency**: Frequency of sample collection (monthly, quarterly, etc.).

### Environmental and Physical Features
- **Weather**: Weather conditions at the time of sampling.
- **Approx Depth**: Approximate depth of the water body at sampling location.
- **Flow**: Flow condition of water (still, slow, fast).
- **Color**: Visual color of the water.
- **Odor**: Smell of the water, indicating possible contamination.
- **Temperature**: Temperature of the water in degrees Celsius.
- **Turbidity**: Measure of water clarity; higher turbidity indicates more suspended particles.
- **Floating Matter**: Presence of floating waste or debris in water.
- **Human Activities**: Human activities near the sampling site that may affect water quality.

### Chemical Properties
- **pH**: Indicates acidity or alkalinity of water. Neutral pH is 7; safe drinking water usually ranges from 6.5 to 8.5.
- **Conductivity**: Measures water’s ability to conduct electricity, indicating dissolved salts.
- **Dissolved O2**: Amount of oxygen dissolved in water, essential for aquatic life.
- **BOD (Biological Oxygen Demand)**: Amount of oxygen required by microorganisms to decompose organic matter. Higher values indicate higher pollution.
- **COD (Chemical Oxygen Demand)**: Amount of oxygen required to oxidize organic and inorganic substances chemically.
- **Nitrate N**: Concentration of nitrate nitrogen, which may indicate agricultural runoff or sewage contamination.
- **Ammonia N**: Amount of ammonia present, indicating waste contamination.
- **Phosphate**: Phosphate concentration, which can cause algal growth if excessive.
- **Chlorides**: Concentration of chloride salts in water.
- **Sulphate**: Sulphate concentration in water.
- **Sodium**: Sodium concentration.
- **Potassium**: Potassium concentration.
- **Flouride**: Fluoride concentration in water.
- **Boron**: Boron concentration, important for agricultural suitability.

### Biological Properties
- **Total Coliform**: Total count of coliform bacteria, indicating possible contamination.
- **Fecal Coliform**: Bacteria originating from fecal matter, indicating sewage contamination.
- **Fecal Streptococci**: Another bacterial indicator of fecal contamination.

### Solid and Mineral Content
- **Total Dissolved Solids**: Total amount of dissolved substances in water.
- **Total Suspended Solids**: Particles suspended in water.
- **Total Fixed Solids**: Fixed solid particles remaining after evaporation.
- **Total Kjeldahl N**: Measure of organic nitrogen and ammonia.

### Alkalinity and Hardness
- **Phenophelene Alkanity**: Partial alkalinity measurement.
- **Total Alkalinity**: Total buffering capacity of water.
- **Hardness CaCo3**: Total hardness caused by calcium and magnesium.
- **Calcium CaCo3**: Calcium hardness concentration.
- **Magnesium CaCo3**: Magnesium hardness concentration.

### Target / Classification Feature
- **Use Based Class**: Classification of water quality based on its suitability for uses such as drinking, irrigation, or industrial purposes.

These features collectively help assess water quality, identify pollution sources, and support water quality classification using machine learning models.

In [ ]:
##checking unique values in evry col
for col in df.columns:
    print(f"\nColumn: {col}")
    print(df[col].unique())

In [ ]:
#count of unique vlaues
df.nunique().sort_values()

In [ ]:
# available values in target function
print(df['Use Based Class'].unique())

In [ ]:
# simplify class labels
df['Water Class'] = df['Use Based Class'].str.extract(r'^(A|B|C|E)')

# handling missing and unknown('no information)
df['Water Class'] = df['Water Class'].fillna('Unknown')

df['Water Class'].value_counts()

In [ ]:
#visual representation of missing values
sns.heatmap(df.isnull())

In [ ]:
# Missing value 
missing_count = df.isnull().sum()

missing_percent = (df.isnull().sum() / len(df)) * 100

missing_table = pd.DataFrame({
    "Missing Count": missing_count,
    "Missing %": missing_percent
}).sort_values("Missing %", ascending=False)

missing_table

## Missing Values Findings —

- Missing value analysis revealed that several observational and descriptive features, including **Use of Water in Down Stream (100%)**, **Remark (96.8%)**, and **Odor (76.6%)**, have extremely low data availability. Due to insufficient coverage, these features were excluded from further analysis.

- Contextual environmental variables such as **Visibility Effluent Discharge (42.3%)** and **Major Polluting Source (32.9%)** also exhibit substantial missingness, reducing their statistical reliability and limiting their usefulness in quantitative analysis.

- In contrast, the primary **physicochemical water quality indicators** (pH, Dissolved Oxygen, BOD, COD, Conductivity, Nutrients, and Microbial indicators) show minimal missing values (~3%), indicating strong data integrity and making them suitable for exploratory analysis, correlation assessment, and predictive modeling.

- Key **structural and categorical attributes**, including sampling metadata (State, District, Sampling Date, Month) and the target variable (Water Class), contain complete data with no missing values, ensuring consistency for temporal, spatial, and classification analysis.

- The missing data pattern is non-random and appears clustered across specific sampling instances, suggesting operational or measurement-level omissions rather than systematic data corruption.



In [ ]:
# Handeling missing values
# DROP useless columns
drop_cols = [
    " Use of Water in Down Stream",
    "Remark",
    "Odor",
    " Visibility Effluent Discharge",
    "Major Polluting Sources",
    "Floating Matter",
    "Color",
    "Human Activities"
]

df = df.drop(columns=drop_cols)

# segrigating numeric and categorical columns
numeric_cols = df.select_dtypes(include=['float64','int64']).columns
categorical_cols = df.select_dtypes(include=['object']).columns

# Fill numeric columns with median
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())

# Fill categorical columns with "Unknown"
for col in categorical_cols:
    df[col] = df[col].fillna("Unknown")

# Verify no missing values remain
df.isnull().sum().sort_values(ascending=False).head(10)

In [ ]:
# string-based missing values
# detect BDL in all columns using string contains
bdl_count_per_column = df.astype(str).apply(lambda col: col.str.contains("BDL", case=False, na=False).sum())

# show only columns with BDL
bdl_count_per_column[bdl_count_per_column > 0].sort_values(ascending=False)

Findings:

- Multiple water quality parameters contained **BDL (Below Detection Limit)** entries, representing values lower than the measurable threshold rather than true zero or missing values.


In [ ]:
# rows containing BDL anywhere
df[df.astype(str).apply(lambda row: row.str.contains("BDL", case=False, na=False).any(), axis=1)]

In [ ]:
#seperating BDL columns
bdl_cols = df.columns[df.astype(str).apply(lambda col: col.str.contains("BDL", na=False).any())]



print(f"Columns with BDL values: {len(bdl_cols)}")
print(bdl_cols)

In [ ]:
#handeling BDL
# ── half-DL substitution for BDL ───────────────────
import re
import numpy as np

def clean_bdl_column(value):
    if isinstance(value, str):
        match = re.search(r'(\d+\.?\d*)', value)
        if match:
            num = float(match.group())
            return num / 2 if 'BDL' in value else num
        return np.nan
    return value

df_clean = df.copy()

for col in bdl_cols:
    df_clean[col] = df_clean[col].apply(clean_bdl_column)
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

df = df_clean

print("BDL columns after cleaning:")
print(df_clean[bdl_cols].dtypes)
print("\nSample stats:")
print(df_clean[bdl_cols].describe().round(2))



# verify result
df.dtypes

## BDL Handling -

- A total of **17 water quality parameters contained BDL (Below Detection Limit) values**, primarily affecting chemical, nutrient, and microbial indicators such as Nitrate, Phosphate, Fluoride, and Coliform.

- These BDL entries were originally stored as **string values**, which prevented direct numerical analysis and required preprocessing.
BDL values were replaced with half the detection limit, a standard environmental practice to avoid bias while maintaining statistical validity.

- BDL values were systematically converted using the **half detection limit substitution method (DL/2)**, a standard environmental data preprocessing technique, ensuring realistic approximation without introducing analytical bias.

- After preprocessing, all affected columns were successfully converted to **float64 numerical type**, making them suitable for statistical analysis, visualization, and machine learning.

- This transformation improved overall dataset usability by converting previously unusable string-based measurement fields into quantitative variables.

- Final datatype distribution showed that the majority of features are now **numerical (float64)**, confirming the dataset is properly structured for downstream analysis and predictive modeling.

In [ ]:
#type of data available
df.dtypes.value_counts()

In [ ]:
#duplicated values
df.duplicated().sum()

In [ ]:
#fixing latitude and longitude
import re

def parse_coord(val):
    if isinstance(val, str):
        m = re.findall(r'[\d.]+', val)
        if len(m) >= 2:
            return float(m[0]) + float(m[1])/60
        elif len(m) == 1:
            return float(m[0])
    return None

df['lat_clean'] = df['latitude'].apply(parse_coord)
df['lon_clean'] = df['longitude'].apply(parse_coord)

In [ ]:
df[['latitude', 'longitude', 'lat_clean', 'lon_clean']].head(20)

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(df['lon_clean'], df['lat_clean'], alpha=0.5)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Station Locations")
plt.show()

No duplicates are present

In [ ]:
df.columns

In [ ]:
 
safe_limits = {

    'Dissolved O2': ('min', 5),
    'BOD': ('max', 3),
    'Total Coliform': ('max', 50),
    'Fecal Coliform': ('max', 10),
    'Turbidity': ('max', 5),
    'Total Suspended Solids': ('max', 100)

}

violation_df = pd.DataFrame()

for param, (condition, limit) in safe_limits.items():
    
    if condition == 'max':
        violation_df[param] = (df[param] > limit).astype(int)
        
    else:
        violation_df[param] = (df[param] < limit).astype(int)

df['Violation_Count'] = violation_df.sum(axis=1)

df['Violation_Count']

## Regulatory Violation Findings

Violation_Count represents the number of water quality parameters exceeding regulatory safety limits at each monitoring station.

Key observations:

- Violation counts range from 0 to 6 across monitoring stations, indicating varying levels of regulatory non-compliance.
- 28 stations have 0 violations, including all of Kolhapur (8 stations), Ratnagiri (6), Sangli (5), and Raigad (5).
- Several stations show multiple violations (3–5 parameters), indicating significant water quality deterioration at those locations.
- Higher violation counts suggest compounded pollution effects, which increase environmental and public health risk.

Overall, regulatory violations are widespread, highlighting the presence of systemic water quality issues rather than isolated anomalies.

In [ ]:
#outlier detection for important columns
critical_cols = [
    "Dissolved O2",
    "BOD",
    "Total Coliform",
    "Fecal Coliform",
    "Turbidity",
    "Total Suspended Solids"
]

outlier_counts = {}

for col in critical_cols:
    
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    
    outlier_counts[col] = ((df[col] < lower) | (df[col] > upper)).sum()
    
outlier_counts

In [ ]:
outlier_df = pd.DataFrame()

for col in critical_cols:
    
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    
    outlier_df[col] = ((df[col] < lower) | (df[col] > upper)).astype(int)

df['Outlier_Count'] = outlier_df.sum(axis=1)

In [ ]:
sns.histplot(df['Outlier_Count'], bins=5)

plt.title("Outlier Count per Station")
plt.show()

## Outlier Analysis of Critical Water Quality Parameters

Outliers were identified using the IQR method across key pollution indicators to detect abnormal or extreme values that may indicate localized contamination.

- Turbidity shows the highest number of outliers (31), indicating frequent abnormal suspended particle levels in water bodies.
- Total Coliform (23 outliers) and BOD (16 outliers) also exhibit significant deviations, suggesting elevated microbial contamination and organic pollution at multiple stations.
- Fecal Coliform (13 outliers) further confirms biological contamination risk in several monitoring locations.
- Dissolved Oxygen has fewer outliers (10), but low oxygen outliers are environmentally critical as they directly affect aquatic life survival.
- Total Suspended Solids shows relatively fewer outliers (6), indicating suspended solids are less frequently extreme compared to biological and turbidity indicators.

Station-level distribution shows:

- Most stations have 0–1 outlier parameters, indicating generally stable water quality conditions.
- However, a smaller subset of stations exhibit 2–5 outlier parameters, representing localized high-risk pollution zones.
- These stations require focused environmental monitoring, as multiple abnormal parameters indicate compounded water quality degradation.

Overall, outlier analysis highlights specific parameters and locations with abnormal pollution levels, providing early indicators of environmental stress and contamination hotspots.

# Risk Classification — Identifying High-Risk Monitoring Stations

Risk classification combines regulatory violations and statistical outliers to assess pollution severity.

Risk Score = Violation_Count/ len(safe_limits)

Stations are classified as:

• Safe: Risk Score 0-1  
• Moderate Risk: Risk Score 2–3 
• High Risk: Risk Score ≥ 4  



In [ ]:
df['Risk_Score'] = df['Violation_Count'] / len(safe_limits)

In [ ]:
df['Risk_Level'] = pd.cut(

    df['Violation_Count'],
    
    bins=[-1, 1, 3, 10],
    
    labels=['Safe', 'Moderate Risk', 'High Risk']
)

df[['Violation_Count', 'Risk_Score', 'Risk_Level']].head(10)

In [ ]:
sns.countplot(data=df, x='Risk_Level')

plt.title("Water Quality Risk Classification")

plt.show()

## Water Quality Risk Classification

Monitoring stations were classified into Safe, Moderate Risk, and High Risk categories based on the number of regulatory parameter violations.

- A significant portion of stations fall under Moderate Risk and High Risk categories, indicating widespread water quality concerns across monitored locations.
- High Risk stations represent the largest group, suggesting multiple regulatory violations occurring simultaneously at several monitoring points.
- Moderate Risk stations also form a substantial proportion, indicating early-stage pollution or partial regulatory non-compliance.
- Safe stations represent a smaller proportion of the total monitoring network, showing that fully compliant water quality conditions are less common.

Interpretation:

- The high number of Moderate and High Risk stations suggests systemic environmental stress rather than isolated contamination.
- Multiple parameter violations indicate combined effects of organic pollution, microbial contamination, and suspended particles.
- These findings highlight the need for targeted pollution control, stricter monitoring, and remediation efforts in high-risk areas.

Overall, risk classification provides a clear prioritization framework for environmental management and intervention planning.

In [ ]:
df.groupby('Water Class')['Risk_Level'].value_counts(normalize=True)

In [ ]:
ax = sns.countplot(data=df, x='Water Class', order=['A','B','C','E'])

total = len(df)

for container in ax.containers:
    labels = [f'{(v.get_height()/total)*100:.1f}%' for v in container]
    ax.bar_label(container, labels=labels)

plt.title("Water Quality Class Distribution (%)")
plt.show()

## Water Quality Class Distribution

Water bodies are classified based on their designated best use, ranging from Class A (highest quality) to Class E (lowest quality).

Key findings:

- Class A dominates the dataset, accounting for approximately 63.5% of monitoring stations.  
  This indicates that most monitored water bodies are designated for drinking purposes without conventional treatment.

- Class E represents about 8.6% of stations, indicating a notable portion of water bodies suitable only for irrigation, industrial cooling, or controlled waste disposal.

- Class C and Class B are relatively rare, representing 2.7% and 2.3% respectively, suggesting fewer water bodies designated for drinking with treatment or outdoor bathing.

Interpretation:

- The high proportion of Class A stations reflects the critical importance of these water bodies as primary drinking water sources.
- However, classification represents intended use rather than actual compliance, meaning Class A stations may still exhibit regulatory violations as observed in risk analysis.
- The presence of Class E stations confirms that certain water bodies are already degraded and unsuitable for human consumption.

Overall, while most stations are classified as high-quality water sources, further analysis is necessary to assess whether they truly meet regulatory safety standards.

In [ ]:
# District Risk Analysis
district_risk = df.groupby('District')['Risk_Score'] \
                  .mean() \
                  .sort_values(ascending=False)

district_risk.head(25)
 

In [ ]:
district_risk.head(10).plot(kind='bar')

plt.title("Top 10 High Risk Districts")

plt.show()

In [ ]:
df.groupby('District')['Risk_Level'].value_counts(normalize=True).sort_values(ascending=False).head(10)

In [ ]:
df.groupby('District')['Risk_Level'].value_counts(normalize=True).sort_values(ascending=True).head(10)

## District-wise Water Quality Risk Analysis

District-level analysis was performed using both average Risk Score and proportional Risk Level distribution to identify geographic pollution hotspots and compliance patterns.

### High Risk Districts (Based on Risk Score)

- Nashik (0.70), Mumbai (0.65), and Pune (0.63) show the highest average risk scores, indicating widespread regulatory violations.
- Bhandara, Solapur, Dhule, Nagpur, Akola, Thane, and Chandrapur also exhibit elevated risk scores above 0.50, confirming persistent pollution stress.

These districts represent critical environmental risk zones requiring priority monitoring and intervention.

### Districts with Predominantly High Risk Stations

Risk level proportion analysis reveals:

- Mumbai has approximately 91.7% High Risk stations, indicating severe and widespread regulatory non-compliance.
- Nashik and Dhule also show strong dominance of High Risk stations, confirming systemic water quality degradation.
- Pune shows a mixed but concerning distribution, with both Moderate and High Risk stations present in significant proportions.

This indicates consistent pollution presence across monitoring stations in major urban and industrial districts.

### Districts with Predominantly Safe Stations

Several districts show complete or near-complete regulatory compliance:

- Kolhapur, Nanded, Parbhani, Beed, Latur, Sangli, Jalna, and Ch. Sambhaji Nagar show 100% Safe stations.
- These districts demonstrate strong compliance with water quality safety thresholds.

These areas represent low-risk zones with stable water quality conditions.

### Mixed Risk Districts

Some districts such as Chandrapur, Nagpur, Satara, Ratnagiri, and Raigad show mixed risk distributions, indicating localized pollution issues rather than widespread contamination.

### Overall Interpretation

- Major urban and industrial districts show the highest pollution risk, likely due to industrial discharge, sewage contamination, and urban runoff.
- Rural and less industrialized districts show better regulatory compliance.
- Risk distribution confirms that water quality degradation is geographically concentrated rather than uniformly distributed.

This analysis provides clear geographic prioritization for environmental regulation, pollution control measures, and water quality management interventions.

In [ ]:
#correlation analysis
sns.heatmap(df[critical_cols].corr(), annot=True)

plt.title("Correlation Matrix")

plt.show()

## Correlation Analysis of Critical Water Quality Parameters

Correlation analysis was performed to understand relationships between key pollution indicators and identify underlying environmental patterns.

### Strong Negative Correlations

- Dissolved Oxygen shows strong negative correlation with:
  - BOD (-0.62)
  - Total Suspended Solids (-0.58)
  - Total Coliform (-0.55)
  - Fecal Coliform (-0.35)

This confirms that as pollution indicators increase, dissolved oxygen levels decrease significantly. This relationship is environmentally critical, as lower oxygen levels directly harm aquatic life and indicate organic pollution.

### Strong Positive Correlations

- Total Coliform and Fecal Coliform show strong positive correlation (0.61), indicating both originate from similar contamination sources, primarily sewage and biological waste.
- Total Suspended Solids shows moderate positive correlation with Fecal Coliform (0.28), BOD (0.24), and Turbidity (0.23), suggesting these parameters increase together during pollution events.

### Weak Correlations

- Turbidity shows relatively weak correlation with most biological indicators, indicating suspended particles may originate from both natural sediment and pollution sources.
- BOD shows only moderate correlation with microbial indicators, suggesting organic pollution may arise from multiple independent sources.

### Key Interpretation

- Dissolved Oxygen acts as a primary indicator of water quality degradation and responds strongly to increases in organic and microbial pollution.
- Microbial contamination indicators (Total Coliform and Fecal Coliform) are closely related, confirming biological pollution patterns.
- Suspended solids contribute to overall pollution but are less directly linked to microbial contamination compared to biological indicators.

Overall, correlation analysis confirms that organic pollution, microbial contamination, and suspended particles collectively contribute to water quality deterioration, with Dissolved Oxygen serving as a critical environmental health indicator.

Business Question

Do different water classes have significantly different Dissolved Oxygen levels?

Null Hypothesis (H₀): Mean DO is same across all water classes

Alternative Hypothesis (H₁): Mean DO differs across water classes

In [ ]:
from scipy.stats import f_oneway

# Extract DO values by class
DO_A = df[df['Water Class'] == 'A']['Dissolved O2'].dropna()
DO_B = df[df['Water Class'] == 'B']['Dissolved O2'].dropna()
DO_C = df[df['Water Class'] == 'C']['Dissolved O2'].dropna()
DO_E = df[df['Water Class'] == 'E']['Dissolved O2'].dropna()

# Perform ANOVA
anova_result = f_oneway(DO_A, DO_B, DO_C, DO_E)

print("ANOVA F-statistic:", anova_result.statistic)
print("P-value:", anova_result.pvalue)

## ANOVA Test: Dissolved Oxygen Across Water Classes

To determine whether Dissolved Oxygen (DO) levels differ significantly across Water Classes, a one-way ANOVA test was performed.

Hypotheses:

- Null Hypothesis (H₀): Mean Dissolved Oxygen levels are the same across all Water Classes.
- Alternative Hypothesis (H₁): Mean Dissolved Oxygen levels differ across Water Classes.

### Test Results

- F-statistic: 0.4705  
- p-value: 0.7032  

### Interpretation

The p-value (0.7032) is significantly greater than the standard significance level of 0.05. Therefore, we fail to reject the null hypothesis.

This indicates that there is no statistically significant difference in mean Dissolved Oxygen levels across different Water Classes.

### Business Interpretation

Although water bodies are classified based on intended use, Dissolved Oxygen levels do not differ significantly between these classes. This suggests that classification alone may not fully reflect actual oxygen conditions, and other pollution indicators such as BOD and microbial contamination may play a more dominant role in determining water quality risk.

### Conclusion

Water Class designation does not show a statistically significant impact on Dissolved Oxygen levels in this dataset. Continuous monitoring of multiple pollution parameters is necessary to accurately assess water quality conditions.

Business Question

Is there a statistically significant relationship between pollution and oxygen levels?

H₀: No correlation between BOD and DO

H₁: Significant correlation exists

In [ ]:
from scipy.stats import pearsonr

corr_df = df[['BOD', 'Dissolved O2']].dropna()

corr, p_value = pearsonr(
    corr_df['BOD'],
    corr_df['Dissolved O2']
)

print("Correlation coefficient:", round(corr, 4))
print("P-value:", p_value)

## Correlation Analysis: BOD vs Dissolved Oxygen

To evaluate the relationship between organic pollution and oxygen availability, a Pearson correlation test was performed between Biochemical Oxygen Demand (BOD) and Dissolved Oxygen (DO).

Hypotheses:

- Null Hypothesis (H₀): No correlation exists between BOD and Dissolved Oxygen.
- Alternative Hypothesis (H₁): A significant correlation exists between BOD and Dissolved Oxygen.

### Test Results

- Correlation coefficient (r): -0.623  
- P-value: 1.645 × 10⁻²⁴  

### Interpretation

The correlation coefficient of -0.623 indicates a strong negative relationship between BOD and Dissolved Oxygen.

The extremely small p-value (< 0.05) confirms that this relationship is statistically significant.

This means that as BOD increases, Dissolved Oxygen levels decrease significantly.

### Environmental Interpretation

Higher BOD indicates increased organic pollution, which consumes oxygen during microbial decomposition. This reduces the amount of dissolved oxygen available in water, negatively impacting aquatic life.

This result confirms that BOD is a critical indicator of water pollution and ecosystem stress.

### Business / Policy Insight

Controlling BOD levels is essential to maintain healthy oxygen levels in water bodies. Monitoring and reducing organic pollution sources such as sewage discharge and industrial waste should be a priority to improve water quality.

Business Question

Does water quality vary significantly across districts?

H₀: Water Class is independent of District

H₁: Water Class depends on District

In [ ]:
from scipy.stats import chi2_contingency


contingency = pd.crosstab(df['District'], df['Water Class'])

chi2, p, dof, expected = chi2_contingency(contingency)

print("Chi-square statistic:", chi2)
print("P-value:", p)

## District vs Water Quality — Chi-Square Test of Independence

To determine whether water quality varies significantly across districts, a Chi-square test of independence was performed between District and Water Class.

### Hypotheses

- Null Hypothesis (H₀): Water Class is independent of District.
- Alternative Hypothesis (H₁): Water Class depends on District.

### Test Results

- Chi-square statistic: 527.22  
- P-value: 8.19 × 10⁻⁶⁴  

### Interpretation

The extremely small p-value (< 0.05) indicates strong statistical evidence to reject the null hypothesis.

This confirms that water quality distribution is significantly different across districts.

In other words, district location has a major influence on water quality.

### Practical Insight

Water quality is not uniform across regions. Some districts consistently show higher pollution levels, while others maintain safer water conditions.

This variation may be due to factors such as:

- Industrial activity
- Urbanization and sewage discharge
- Waste management practices
- Population density

### Business / Policy Insight

Water quality management should be prioritized at the district level. High-risk districts require targeted intervention, stricter pollution control, and continuous monitoring to improve environmental and public health outcomes.

Business Question

Which districts have worst water quality?


In [ ]:
district_risk = df.groupby('District')['Risk_Score'].mean().sort_values(ascending=False)

top10 = district_risk.head(10)

plt.figure(figsize=(10,6))
sns.barplot(x=top10.values, y=top10.index)

plt.title("Top 10 High-Risk Districts Based on Water Quality Violations")
plt.xlabel("Average Risk Score")
plt.ylabel("District")

plt.show()
 


Nashik, Mumbai, and Pune have the highest risk scores, indicating the greatest proportion of water quality violations.

These districts likely face higher pollution pressure from urbanization, industrial discharge, and sewage contamination.

These areas should be prioritized for intervention and monitoring.


## Executive Summary

This confirms pollution is **geographically clustered**, not random.

### Highest Risk Districts

| District | Risk Score | Primary Drivers |
|--------|------------|----------------|
| 🔴 Nashik | Highest | Industrial and domestic waste |
| 🔴 Mumbai | High | Industrial discharge, MIDC zones |
| 🔴 Pune | High | Urban sewage and industrial zones |
| 🔴 Nagpur | High | Industrial discharge, urban sewage |


### Lowest Risk Districts

| District | Risk Level |
|--------|-------------|
| 🟢 Kolhapur | Low |
| 🟢 Ratnagiri | Low |
| 🟢 Beed | Low |
| 🟢 Sangli | Low |

Pollution is concentrated in **industrial and urban districts**, especially MIDC zones.

---

##  Business & Policy Implications

### 1. Priority Intervention in High-Risk Districts
Immediate remediation required in:
- Nashik  
- Mumbai 
- Pune  
- Nagpur


Industrial discharge monitoring must be strengthened.

---

### 2. Water Classification Audit Required

Current classifications do not reflect actual safety levels.

Stations classified as Class A but violating BIS thresholds must be re-evaluated.

---

### 3. Public Health Risk from Microbial Contamination

With **79% of stations exceeding coliform limits**, untreated sewage is a major public health concern.

Priority investment needed in sewage treatment infrastructure.

---

### 4. Risk-Based Monitoring Strategy

Chi-Square results confirm pollution is geographically concentrated.

Monitoring should prioritize high-risk districts rather than uniform sampling.

This improves resource efficiency and surveillance effectiveness.

---

##  Final Conclusion

This analysis reveals a critical gap between **water classification labels and actual regulatory compliance.**

Although **63.5% of stations are classified as Class A**, nearly:

- **4 out of 5 stations fail BOD or coliform safety thresholds**
- Organic pollution and microbial contamination are the primary drivers
- Pollution is concentrated in industrial and urban districts

This confirms that Maharashtra’s water quality challenge is not monitoring — it is pollution control and enforcement.

---

##  Final Insight

> Maharashtra does not have a water monitoring problem.  
> Maharashtra has a pollution control and enforcement problem — and data has now made it measurable.

In [ ]:
#saving in csv file
df.to_csv("water_quality_processed.csv", index=False)